In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
INF = 1 << 30
class MCMF:
  def __init__(self, N, src, snk) :
    self.len = N
    self.G = [[] for _ in range(N)]
    self.C = [[0] * N for _ in range(N)]
    self.F = [[0] * N for _ in range(N)]
    self.W = [[0] * N for _ in range(N)]
    self.chk = [False] * N
    self.vis = [False] * N
    self.lvl = [0] * N
    self.src = src
    self.snk = snk
    self.mc = 0

  def add(self, u, v, w, c) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def bfs(self) :
    Q = collections.deque([self.src])
    inQ = [False] * self.len 
    lvl = [INF] * self.len
    lvl[self.src] = 0

    while Q :
      u = Q.popleft()
      inQ[u] = False

      for v in self.G[u] :
        if self.C[u][v] > self.F[u][v] and lvl[v] > lvl[u] + self.W[u][v] :
          lvl[v] = lvl[u] + self.W[u][v]
          if not inQ[v] :
            Q.append(v)
            inQ[v] = True
    
    self.lvl = lvl
    return self.lvl[self.snk] != INF

  def dfs(self, u, w) :
    if u == self.snk : return w
    flow = w
    self.vis[u] = True

    for v in self.G[u] :
      if not flow : break
      if not self.vis[v] and not self.chk[v] and self.C[u][v] - self.F[u][v] > 0 and self.lvl[v] == self.lvl[u] + self.W[u][v] :
        res = self.dfs(v, min(flow, self.C[u][v] - self.F[u][v]))
        if res == 0 : 
          self.chk[v] = True
          continue
        self.F[u][v] += res
        self.F[v][u] -= res
        self.mc += res * self.W[u][v]
        flow -= res
    
    self.vis[u] = False
    return w - flow

  def calc(self) :
    mf = 0
    while self.bfs() :
      self.chk = [False] * self.len
      self.vis = [False] * self.len
      mf += self.dfs(self.src, INF)
    return self.mc, mf

def sol(N, E) :
  V = N * 2 + 3
  src, snk = V-2, V-1
  G = MCMF(V, src, snk)
  G.add(src, 1, 0, 2)
  G.add(N, snk, 0, 2)
  
  for i in range(N) : #정점 분리
    if i in (1, N): G.add(i, i + N, 0, 2) # 분리된 정점에서 1번과 V번 경로는 최대 2번까지 선택 가능하다 <-> cap을 2로 설정
    else : G.add(i, i + N, 0, 1)
  
  for _ in range(E) :
    u, v, w = map(int, input().split())
    G.add(u + N, v, w, 1)
  
  mc, mf = G.calc()
  return mc

ans = []
while True :
  try :
    n, e = map(int, input().split())
    ans.append(sol(n, e))
  except :
    break

print(*ans, sep='\n')

- 플로우 구조를 너무 2단계 병렬구조로만 생각해서 답이 안나왔던것 같다.
- 여러개의 직렬-병렬 구조로 구성된 설계도 생각했어야 했다.
- u,v,c,w로 입력받는거에서 u,v,w,c 순서로 입력받도록 변경
- 정점분리 규칙 및 src, snk 규칙도 적어놔야겟다